# Crime Data Analysis and Classification

## Overview:

This script processes crime data to assign danger levels to different time and precinct combinations, trains a Gradient Boosting Classifier to predict danger levels, and evaluates the model's performance. The code includes the following steps:

1. Exploring dataset.
2. Cleaning dataset.
3. Analysing dataset.
4. Mapping weights to crime categories.
5. Aggregating the data by time and precinct.
6. Defining danger levels based on crime count thresholds.
7. Preparing the data for training.
8. Training a Gradient Boosting Classifier using GridSearchCV
9. Evaluating the model's performance.

## Requirements:

### The following Python packages are required to run this script:
- numpy
- pandas
- scikit-learn

## Running:

To run the code you need to have dataset downloaded from: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i, under the name: "NYPD_Complaint_Data_Historic.csv" in the directory with the code and then run all the cells below.

## 1. Importing Libraries:


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

pandas, numpy: For data manipulation and analysis.

train_test_split, GridSearchCV: For splitting the data and hyperparameter tuning.

GradientBoostingClassifier: The machine learning model used.

classification_report: For evaluating model performance.

## 2. Exploring Dataset:

In [ ]:
# API
# import requests
# import json
#
# url = "https://data.cityofnewyork.us/resource/qgea-i56i.json?"
#
# response = requests.get(url)
#
# if response.status_code == 200:
#     data = response.json()
#     file_path = "NYPD_Complaint_Data_Historic.json"
#     with open(file_path, "w") as file:
#         json.dump(data, file)
#     file_path
# else:
#     response.status_code
# import pandas as pd
#
# file_path = "NYPD_Complaint_Data_Historic.json"
# with open(file_path, "r") as file:
#     data = json.load(file)
#
# df = pd.DataFrame(data)

Sends a GET request to the New York City Open Data API to fetch historical NYPD complaint data, however we did not use it as it is limited to get only 1000 rows per request

In [2]:
df = pd.read_csv("NYPD_Complaint_Data_Historic.csv", sep = ",")

/var/folders/xk/_55wh_553_53h3nkb0_2byvw0000gn/T/ipykernel_29276/3086326548.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("NYPD_Complaint_Data_Historic.csv", sep = ",")


Reads the historical NYPD complaint data from a CSV file into a pandas DataFrame

In [3]:
print("First 10 rows of the data:")
print(df.head(10))

First 10 rows of the data:
  CMPLNT_NUM CMPLNT_FR_DT CMPLNT_FR_TM CMPLNT_TO_DT CMPLNT_TO_TM  ADDR_PCT_CD  \
0   39468181   02/20/2008     07:00:00   02/23/2008     08:00:00         88.0   
1   50539499   08/21/2008     22:00:00   08/21/2008     23:00:00         19.0   
2   45223390   04/03/2008     03:35:00   04/03/2008     03:50:00         77.0   
3   50594658   08/19/2008     09:00:00          NaN       (null)         32.0   
4   44451016   03/10/2008     22:00:00   03/10/2008     22:10:00         67.0   
5   49918184   08/08/2008     01:30:00   08/08/2008     01:34:00         34.0   
6   55066472   12/21/2008     06:07:00          NaN       (null)         78.0   
7   55244936   12/27/2008     02:04:00          NaN       (null)         23.0   
8   47391386   06/19/2008     15:30:00   06/19/2008     16:30:00         61.0   
9   45814265   04/19/2008     19:45:00   04/19/2008     19:50:00         83.0   

       RPT_DT  KY_CD                       OFNS_DESC  PD_CD  ... SUSP_SEX  \
0  0

Prints first 10 rows

In [4]:
print("Shape of the data:", df.shape)

Shape of the data: (8914838, 35)


Prints shape of the data

In [5]:
print("Columns in the data:", df.columns)

Columns in the data: Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD',
       'PD_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'BORO_NM',
       'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'JURIS_DESC', 'JURISDICTION_CODE',
       'PARKS_NM', 'HADEVELOPT', 'HOUSING_PSA', 'X_COORD_CD', 'Y_COORD_CD',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT',
       'Latitude', 'Longitude', 'Lat_Lon', 'PATROL_BORO', 'STATION_NAME',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX'],
      dtype='object')


Prints columns of the data

In [6]:
print("Descriptive statistics of the data:")
print(df.describe())

Descriptive statistics of the data:
        ADDR_PCT_CD         KY_CD         PD_CD  JURISDICTION_CODE  \
count  8.914067e+06  8.914838e+06  8.907245e+06       8.914838e+06   
mean   6.336865e+01  2.972099e+02  4.124253e+02       6.833533e-01   
std    3.459529e+01  1.521695e+02  2.181523e+02       6.498413e+00   
min    1.000000e+00  1.010000e+02  1.000000e+02       0.000000e+00   
25%    4.000000e+01  1.170000e+02  2.540000e+02       0.000000e+00   
50%    6.300000e+01  3.410000e+02  3.610000e+02       0.000000e+00   
75%    9.400000e+01  3.510000e+02  6.370000e+02       0.000000e+00   
max    1.230000e+02  8.810000e+02  9.750000e+02       9.700000e+01   

         X_COORD_CD    Y_COORD_CD  TRANSIT_DISTRICT      Latitude  \
count  8.914372e+06  8.914372e+06     198045.000000  8.914372e+06   
mean   1.004946e+06  2.072020e+05         13.843293  4.073530e+01   
std    2.150845e+04  3.035500e+04         12.541970  9.481303e-02   
min    0.000000e+00  0.000000e+00          1.000000  0.00

Prints descriptive statistics

In [7]:
print("Info about the data:")
df.info()

Info about the data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914838 entries, 0 to 8914837
Data columns (total 35 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CMPLNT_NUM         object 
 1   CMPLNT_FR_DT       object 
 2   CMPLNT_FR_TM       object 
 3   CMPLNT_TO_DT       object 
 4   CMPLNT_TO_TM       object 
 5   ADDR_PCT_CD        float64
 6   RPT_DT             object 
 7   KY_CD              int64  
 8   OFNS_DESC          object 
 9   PD_CD              float64
 10  PD_DESC            object 
 11  CRM_ATPT_CPTD_CD   object 
 12  LAW_CAT_CD         object 
 13  BORO_NM            object 
 14  LOC_OF_OCCUR_DESC  object 
 15  PREM_TYP_DESC      object 
 16  JURIS_DESC         object 
 17  JURISDICTION_CODE  int64  
 18  PARKS_NM           object 
 19  HADEVELOPT         object 
 20  HOUSING_PSA        object 
 21  X_COORD_CD         float64
 22  Y_COORD_CD         float64
 23  SUSP_AGE_GROUP     object 
 24  SUSP_RACE          object 
 2

Prints information about the data

In [8]:
print("Data types of each column:")
for col in df.columns:
    t = set()
    for item in df[col]:
        t.add(type(item))
    print(f"Column '{col}' has types: {t}")

Data types of each column:
Column 'CMPLNT_NUM' has types: {<class 'str'>, <class 'int'>}
Column 'CMPLNT_FR_DT' has types: {<class 'float'>, <class 'str'>}
Column 'CMPLNT_FR_TM' has types: {<class 'str'>}
Column 'CMPLNT_TO_DT' has types: {<class 'float'>, <class 'str'>}
Column 'CMPLNT_TO_TM' has types: {<class 'str'>}
Column 'ADDR_PCT_CD' has types: {<class 'float'>}
Column 'RPT_DT' has types: {<class 'str'>}
Column 'KY_CD' has types: {<class 'int'>}
Column 'OFNS_DESC' has types: {<class 'str'>}
Column 'PD_CD' has types: {<class 'float'>}
Column 'PD_DESC' has types: {<class 'str'>}
Column 'CRM_ATPT_CPTD_CD' has types: {<class 'str'>}
Column 'LAW_CAT_CD' has types: {<class 'str'>}
Column 'BORO_NM' has types: {<class 'str'>}
Column 'LOC_OF_OCCUR_DESC' has types: {<class 'float'>, <class 'str'>}
Column 'PREM_TYP_DESC' has types: {<class 'str'>}
Column 'JURIS_DESC' has types: {<class 'str'>}
Column 'JURISDICTION_CODE' has types: {<class 'int'>}
Column 'PARKS_NM' has types: {<class 'str'>}
C

Prints data types of each column

In [9]:
print("Unique values in each column:")
for col in df.columns:
    print(f"Column '{col}' unique values:", df[col].unique())

Unique values in each column:
Column 'CMPLNT_NUM' unique values: [39468181 50539499 45223390 ... 49520303 47521554 51084712]
Column 'CMPLNT_FR_DT' unique values: ['02/20/2008' '08/21/2008' '04/03/2008' ... '01/20/2004' '11/13/1998'
 '08/05/1999']
Column 'CMPLNT_FR_TM' unique values: ['07:00:00' '22:00:00' '03:35:00' ... '00:00:01' '05:59:00' '06:31:00']
Column 'CMPLNT_TO_DT' unique values: ['02/23/2008' '08/21/2008' '04/03/2008' ... '07/07/1999' '06/16/2003'
 '07/31/1994']
Column 'CMPLNT_TO_TM' unique values: ['08:00:00' '23:00:00' '03:50:00' ... '05:54:00' '07:48:00' '06:53:00']
Column 'ADDR_PCT_CD' unique values: [ 88.  19.  77.  32.  67.  34.  78.  23.  61.  83.  66.  62.  20.  44.
  17.  70.  73.  63.  46.  40.  28.  79.  26.  18.  24.  71.  43.  33.
  90. 114.  69.  30. 108.  81.  50.  94.  25.  60.  42.   9.  13. 104.
  72.  22.  52.   7. 115.  49.  47. 100.  41. 112.  75.  48. 110. 102.
 106. 109.  45. 107. 103. 113. 105. 111. 101.   1. 120.  10. 122.  68.
   6. 123.  nan  76.  

Prints unique values for each column

In [10]:
print("Percentage of missing values for each column:")
missing_percentage = (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False)
print(missing_percentage, "\n")

Percentage of missing values for each column:
TRANSIT_DISTRICT     97.778479
CMPLNT_TO_DT         20.403713
PD_CD                 0.085173
ADDR_PCT_CD           0.008649
CMPLNT_FR_DT          0.007347
Longitude             0.005227
Y_COORD_CD            0.005227
Latitude              0.005227
X_COORD_CD            0.005227
Lat_Lon               0.005227
LOC_OF_OCCUR_DESC     0.002625
HOUSING_PSA           0.000000
PATROL_BORO           0.000000
STATION_NAME          0.000000
SUSP_SEX              0.000000
SUSP_RACE             0.000000
SUSP_AGE_GROUP        0.000000
VIC_AGE_GROUP         0.000000
VIC_RACE              0.000000
CMPLNT_NUM            0.000000
JURISDICTION_CODE     0.000000
HADEVELOPT            0.000000
PARKS_NM              0.000000
JURIS_DESC            0.000000
PREM_TYP_DESC         0.000000
BORO_NM               0.000000
LAW_CAT_CD            0.000000
CRM_ATPT_CPTD_CD      0.000000
PD_DESC               0.000000
OFNS_DESC             0.000000
KY_CD                 0.

Prints the percentage of missing values for each column

## 3. Cleaning Dataset:

In [11]:
df.drop(["CRM_ATPT_CPTD_CD", "HADEVELOPT", "JURISDICTION_CODE", "JURIS_DESC", "KY_CD", "PARKS_NM",
         "PD_DESC", "RPT_DT", "X_COORD_CD", "Y_COORD_CD", "TRANSIT_DISTRICT"], axis="columns", inplace=True)

Drops specified columns that are unnecessary or contain redundant information

In [12]:
df['CMPLNT_NUM'] = df['CMPLNT_NUM'].astype(str).str.strip()

Converts the 'CMPLNT_NUM' column to a string and strip any leading or trailing whitespace

In [13]:
df["CMPLNT_FR_DT"] = pd.to_datetime(df["CMPLNT_FR_DT"], format="%m/%d/%Y", errors="coerce")
df["CMPLNT_TO_DT"] = pd.to_datetime(df["CMPLNT_TO_DT"], format="%m/%d/%Y", errors="coerce")

Converts complaint date columns to datetime objects

In [14]:
df["CMPLNT_FR_TM"] = pd.to_datetime(df["CMPLNT_FR_TM"], errors="coerce")
df["CMPLNT_TO_TM"] = pd.to_datetime(df["CMPLNT_TO_TM"], errors="coerce")

/var/folders/xk/_55wh_553_53h3nkb0_2byvw0000gn/T/ipykernel_29276/495979590.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["CMPLNT_FR_TM"] = pd.to_datetime(df["CMPLNT_FR_TM"], errors="coerce")
/var/folders/xk/_55wh_553_53h3nkb0_2byvw0000gn/T/ipykernel_29276/495979590.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["CMPLNT_TO_TM"] = pd.to_datetime(df["CMPLNT_TO_TM"], errors="coerce")


Converts complaint time columns to time objects in 'HH:MM:SS' format

In [15]:
df['ADDR_PCT_CD'] = pd.to_numeric(df['ADDR_PCT_CD'], errors='coerce').fillna(-1).astype(int)
df['PD_CD'] = pd.to_numeric(df['PD_CD'], errors='coerce').fillna(-1).astype(int)
df['HOUSING_PSA'] = pd.to_numeric(df['HOUSING_PSA'], errors='coerce').fillna(-1).astype(int)
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

Converts numeric columns to appropriate data types, replacing invalid values with -1 for integers and NaN for floats

In [16]:
df['BORO_NM'] = df['BORO_NM'].replace({'(null)': np.nan}).str.upper().str.strip()
df['LOC_OF_OCCUR_DESC'] = df['LOC_OF_OCCUR_DESC'].replace({'(null)': np.nan}).str.upper().str.strip()
df['PATROL_BORO'] = df['PATROL_BORO'].str.upper().str.strip()
df['PREM_TYP_DESC'] = df['PREM_TYP_DESC'].str.upper().str.strip().fillna('UNKNOWN')

Cleans and formats various string columns by converting to uppercase, stripping whitespace, and handling null values

In [17]:
valid_age_groups = ['25-44', '45-64', '18-24', '<18', 'UNKNOWN', '65+']
df['SUSP_AGE_GROUP'] = df['SUSP_AGE_GROUP'].str.upper().str.strip()
df['SUSP_AGE_GROUP'] = df['SUSP_AGE_GROUP'].replace('(null)', np.nan)
df['SUSP_AGE_GROUP'] = df['SUSP_AGE_GROUP'].where(df['SUSP_AGE_GROUP'].isin(valid_age_groups), np.nan)
df['VIC_AGE_GROUP'] = df['VIC_AGE_GROUP'].str.upper().str.strip()
df['VIC_AGE_GROUP'] = df['VIC_AGE_GROUP'].replace('(null)', np.nan)
df['VIC_AGE_GROUP'] = df['VIC_AGE_GROUP'].where(df['VIC_AGE_GROUP'].isin(valid_age_groups), np.nan)

In [18]:
valid_races = ['WHITE', 'WHITE HISPANIC', 'BLACK', 'UNKNOWN', 'ASIAN / PACIFIC ISLANDER', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE']
df['SUSP_RACE'] = df['SUSP_RACE'].str.upper().str.strip()
df['SUSP_RACE'] = df['SUSP_RACE'].replace('(null)', np.nan)
df['SUSP_RACE'] = df['SUSP_RACE'].where(df['SUSP_RACE'].isin(valid_races), np.nan)
df['VIC_RACE'] = df['VIC_RACE'].str.upper().str.strip()
df['VIC_RACE'] = df['VIC_RACE'].replace('(null)', np.nan)
df['VIC_RACE'] = df['VIC_RACE'].where(df['VIC_RACE'].isin(valid_races), np.nan)

In [19]:
valid_sexes = ['F', 'M']
df['SUSP_SEX'] = df['SUSP_SEX'].str.upper().str.strip()
df['SUSP_SEX'] = df['SUSP_SEX'].replace('(null)', np.nan)
df['SUSP_SEX'] = df['SUSP_SEX'].where(df['SUSP_SEX'].isin(valid_sexes), np.nan)
df['VIC_SEX'] = df['VIC_SEX'].str.upper().str.strip()
df['VIC_SEX'] = df['VIC_SEX'].replace('(null)', np.nan)
df['VIC_SEX'] = df['VIC_SEX'].where(df['VIC_SEX'].isin(valid_sexes), np.nan)

Maps categorical columns to specified valid categories or mark them as NaN if they don't fit

In [20]:
offense_mapping = {
    'BURGLARY': 'PROPERTY CRIMES',
    'GRAND LARCENY': 'PROPERTY CRIMES',
    'FELONY ASSAULT': 'VIOLENT CRIMES',
    'PETIT LARCENY': 'PROPERTY CRIMES',
    'ROBBERY': 'VIOLENT CRIMES',
    'OFFENSES AGAINST THE PERSON': 'VIOLENT CRIMES',
    'ASSAULT 3 & RELATED OFFENSES': 'VIOLENT CRIMES',
    'INTOXICATED & IMPAIRED DRIVING': 'TRAFFIC OFFENSES',
    'CRIMINAL TRESPASS': 'PROPERTY CRIMES',
    'HARRASSMENT 2': 'PUBLIC ORDER CRIMES',
    'OTHER OFFENSES RELATED TO THEF': 'PROPERTY CRIMES',
    'DANGEROUS WEAPONS': 'VIOLENT CRIMES',
    'CRIMINAL MISCHIEF & RELATED OF': 'PROPERTY CRIMES',
    'THEFT-FRAUD': 'PROPERTY CRIMES',
    'DANGEROUS DRUGS': 'DRUG OFFENSES',
    'OFF. AGNST PUB ORD SENSBLTY &': 'PUBLIC ORDER CRIMES',
    'OFFENSES AGAINST PUBLIC ADMINI': 'PUBLIC ORDER CRIMES',
    'GRAND LARCENY OF MOTOR VEHICLE': 'PROPERTY CRIMES',
    'MISCELLANEOUS PENAL LAW': 'OTHER OFFENSES',
    'SEX CRIMES': 'SEXUAL OFFENSES',
    'VEHICLE AND TRAFFIC LAWS': 'TRAFFIC OFFENSES',
    'RAPE': 'SEXUAL OFFENSES',
    'ARSON': 'PROPERTY CRIMES',
    'FORGERY': 'FRAUD OFFENSES',
    'GAMBLING': 'PUBLIC ORDER CRIMES',
    'FRAUDS': 'FRAUD OFFENSES',
    "BURGLAR'S TOOLS": 'PROPERTY CRIMES',
    'DISORDERLY CONDUCT': 'PUBLIC ORDER CRIMES',
    'ADMINISTRATIVE CODE': 'OTHER OFFENSES',
    'UNAUTHORIZED USE OF A VEHICLE': 'PROPERTY CRIMES',
    'KIDNAPPING & RELATED OFFENSES': 'VIOLENT CRIMES',
    'NYS LAWS-UNCLASSIFIED FELONY': 'OTHER OFFENSES',
    'OTHER STATE LAWS (NON PENAL LA': 'OTHER OFFENSES',
    'POSSESSION OF STOLEN PROPERTY': 'PROPERTY CRIMES',
    '(null)': 'UNKNOWN',
    'OFFENSES INVOLVING FRAUD': 'FRAUD OFFENSES',
    'PETIT LARCENY OF MOTOR VEHICLE': 'PROPERTY CRIMES',
    'FRAUDULENT ACCOSTING': 'FRAUD OFFENSES',
    'ALCOHOLIC BEVERAGE CONTROL LAW': 'PUBLIC ORDER CRIMES',
    'ESCAPE 3': 'PUBLIC ORDER CRIMES',
    'THEFT OF SERVICES': 'PROPERTY CRIMES',
    'CHILD ABANDONMENT/NON SUPPORT': 'VIOLENT CRIMES',
    'OFFENSES RELATED TO CHILDREN': 'VIOLENT CRIMES',
    'LOITERING': 'PUBLIC ORDER CRIMES',
    'OFFENSES AGAINST PUBLIC SAFETY': 'PUBLIC ORDER CRIMES',
    'ENDAN WELFARE INCOMP': 'VIOLENT CRIMES',
    'PROSTITUTION & RELATED OFFENSES': 'SEXUAL OFFENSES',
    'INTOXICATED/IMPAIRED DRIVING': 'TRAFFIC OFFENSES',
    'OTHER STATE LAWS': 'OTHER OFFENSES',
    'LOITERING/GAMBLING (CARDS, DIC': 'PUBLIC ORDER CRIMES',
    'ADMINISTRATIVE CODES': 'OTHER OFFENSES',
    'HOMICIDE-NEGLIGENT-VEHICLE': 'VIOLENT CRIMES',
    'ANTICIPATORY OFFENSES': 'OTHER OFFENSES',
    'HOMICIDE-NEGLIGENT,UNCLASSIFIE': 'VIOLENT CRIMES',
    'LOITERING/DEVIATE SEX': 'SEXUAL OFFENSES',
    'LOITERING FOR DRUG PURPOSES': 'DRUG OFFENSES',
    'JOSTLING': 'PROPERTY CRIMES',
    'UNLAWFUL POSS. WEAP. ON SCHOOL': 'VIOLENT CRIMES',
    'NEW YORK CITY HEALTH CODE': 'OTHER OFFENSES',
    'OTHER TRAFFIC INFRACTION': 'TRAFFIC OFFENSES',
    'MURDER & NON-NEGL. MANSLAUGHTER': 'VIOLENT CRIMES',
    'DISRUPTION OF A RELIGIOUS SERV': 'PUBLIC ORDER CRIMES',
    'UNDER THE INFLUENCE OF DRUGS': 'DRUG OFFENSES',
    'OFFENSES AGAINST MARRIAGE UNCL': 'OTHER OFFENSES',
    'FORTUNE TELLING': 'PUBLIC ORDER CRIMES',
    'OTHER STATE LAWS (NON PENAL LAW)': 'OTHER OFFENSES',
    'CRIMINAL MISCHIEF & RELATE': 'PROPERTY CRIMES'
}
df['OFNS_DESC'] = df['OFNS_DESC'].str.upper().str.strip()
df['OFNS_DESC'] = df['OFNS_DESC'].map(offense_mapping)

Maps offense descriptions to broader categories defined in the offense_mapping dictionary

In [21]:
premise_mapping = {
    'RESIDENCE - APT. HOUSE': 'RESIDENTIAL',
    'BAR/NIGHT CLUB': 'COMMERCIAL',
    'RESIDENCE - PUBLIC HOUSING': 'RESIDENTIAL',
    'STREET': 'PUBLIC SPACE',
    'VARIETY STORE': 'COMMERCIAL',
    'RESIDENCE-HOUSE': 'RESIDENTIAL',
    'TAXI (YELLOW LICENSED)': 'TRANSPORT',
    'COMMERCIAL BUILDING': 'COMMERCIAL',
    'PUBLIC BUILDING': 'PUBLIC SPACE',
    'OTHER': 'OTHER',
    'BANK': 'COMMERCIAL',
    'GROCERY/BODEGA': 'COMMERCIAL',
    'COLLEGE/UNIVERSITY': 'EDUCATIONAL',
    'PUBLIC SCHOOL': 'EDUCATIONAL',
    'PARKING LOT/GARAGE (PRIVATE)': 'PUBLIC SPACE',
    'HOTEL/MOTEL': 'COMMERCIAL',
    'CHAIN STORE': 'COMMERCIAL',
    'PARK/PLAYGROUND': 'PUBLIC SPACE',
    'TRANSIT - NYC SUBWAY': 'TRANSPORT',
    'DEPARTMENT STORE': 'COMMERCIAL',
    'OPEN AREAS (OPEN LOTS)': 'PUBLIC SPACE',
    'DRUG STORE': 'COMMERCIAL',
    'TELECOMM. STORE': 'COMMERCIAL',
    'FAST FOOD': 'COMMERCIAL',
    'TRANSIT FACILITY (OTHER)': 'TRANSPORT',
    'CHECK CASHING BUSINESS': 'COMMERCIAL',
    'RESTAURANT/DINER': 'COMMERCIAL',
    'PARKING LOT/GARAGE (PUBLIC)': 'PUBLIC SPACE',
    'GYM/FITNESS FACILITY': 'COMMERCIAL',
    'BUS STOP': 'TRANSPORT',
    'STORE UNCLASSIFIED': 'COMMERCIAL',
    'CANDY STORE': 'COMMERCIAL',
    'GAS STATION': 'COMMERCIAL',
    'FOOD SUPERMARKET': 'COMMERCIAL',
    'SMALL MERCHANT': 'COMMERCIAL',
    'CLOTHING/BOUTIQUE': 'COMMERCIAL',
    'DOCTOR/DENTIST OFFICE': 'MEDICAL',
    'HIGHWAY/PARKWAY': 'PUBLIC SPACE',
    'BEAUTY & NAIL SALON': 'COMMERCIAL',
    'ABANDONED BUILDING': 'OTHER',
    'SYNAGOGUE': 'RELIGIOUS',
    'HOSPITAL': 'MEDICAL',
    'TAXI (LIVERY LICENSED)': 'TRANSPORT',
    'ATM': 'COMMERCIAL',
    'SOCIAL CLUB/POLICY': 'COMMERCIAL',
    'CHURCH': 'RELIGIOUS',
    'CONSTRUCTION SITE': 'OTHER',
    'STORAGE FACILITY': 'COMMERCIAL',
    'PRIVATE/PAROCHIAL SCHOOL': 'EDUCATIONAL',
    'BUS (NYC TRANSIT)': 'TRANSPORT',
    'FACTORY/WAREHOUSE': 'COMMERCIAL',
    'JEWELRY': 'COMMERCIAL',
    'VIDEO STORE': 'COMMERCIAL',
    'DRY CLEANER/LAUNDRY': 'COMMERCIAL',
    'BRIDGE': 'PUBLIC SPACE',
    'LIQUOR STORE': 'COMMERCIAL',
    'TUNNEL': 'PUBLIC SPACE',
    'MOSQUE': 'RELIGIOUS',
    'CEMETERY': 'PUBLIC SPACE',
    'BOOK/CARD': 'COMMERCIAL',
    'BUS (OTHER)': 'TRANSPORT',
    'TRAMWAY': 'TRANSPORT',
    'SHOE': 'COMMERCIAL',
    'LOAN COMPANY': 'COMMERCIAL',
    'OTHER HOUSE OF WORSHIP': 'RELIGIOUS',
    'BUS TERMINAL': 'TRANSPORT',
    'AIRPORT TERMINAL': 'TRANSPORT',
    'PHOTO/COPY': 'COMMERCIAL',
    'MARINA/PIER': 'PUBLIC SPACE',
    'TAXI/LIVERY (UNLICENSED)': 'TRANSPORT',
    'FERRY/FERRY TERMINAL': 'TRANSPORT',
    '(null)': 'UNKNOWN',
    'HOMELESS SHELTER': 'RESIDENTIAL',
    'MULTI DWELL - APT BUILD': 'RESIDENTIAL',
    'PVT HOUSE': 'RESIDENTIAL',
    'MULTI DWELL - PUBLIC HOUS': 'RESIDENTIAL',
    'COMMERCIAL BLDG': 'COMMERCIAL',
    'BEAUTY/NAIL SALON': 'COMMERCIAL',
    'JEWELRY STORE': 'COMMERCIAL',
    'SUPERMARKET': 'COMMERCIAL',
    'SOCIAL CLUB/POLICY LOCATI': 'COMMERCIAL',
    'REAL ESTATE': 'COMMERCIAL',
    'CLOTHING BOUTIQUE': 'COMMERCIAL',
    'DEPT STORE': 'COMMERCIAL',
    'MOBILE FOOD': 'COMMERCIAL',
    'MAILBOX OUTSIDE': 'PUBLIC SPACE',
    'DOCTOR/DENTIST': 'MEDICAL',
    np.nan: 'UNKNOWN',
}

df['PREM_TYP_DESC'] = df['PREM_TYP_DESC'].str.upper().str.strip()
df['PREM_TYP_DESC'] = df['PREM_TYP_DESC'].map(premise_mapping)

Maps premise types to broader categories defined in the premise_mapping dictionary

In [22]:
df['STATION_NAME'] = df['STATION_NAME'].str.upper().str.strip()
df['STATION_NAME'] = np.select(
    [
        df['STATION_NAME'].str.contains('STREET', case=False, na=False),
        df['STATION_NAME'].str.contains('AVENUE', case=False, na=False),
        df['STATION_NAME'].str.contains('ROAD|PKWY|HIGHWAY|BLVD|LANE', case=False, na=False),
        df['STATION_NAME'].str.contains('SQUARE|PLAZA|CENTER|TERMINAL|CIRCLE|COURT', case=False, na=False),
        df['STATION_NAME'].str.contains('HALL|BRIDGE|BUILDING|PIER|TUNNEL|HILL', case=False, na=False),
        df['STATION_NAME'].str.contains('PARK|GARDEN|PARKWAY', case=False, na=False),
        df['STATION_NAME'].str.contains('STATION|AIRPORT', case=False, na=False),
        df['STATION_NAME'].str.contains('OFFICE', case=False, na=False),
        df['STATION_NAME'].str.contains('COLLEGE|UNIVERSITY|SCHOOL', case=False, na=False),
        df['STATION_NAME'].str.contains('HOSPITAL|CLINIC|MEDICAL|HEALTH', case=False, na=False),
        df['STATION_NAME'].str.contains('MUSEUM|AQUARIUM|GALLERY|EXHIBITION|CULTURE', case=False, na=False),
        df['STATION_NAME'].str.contains('SHOPPING|MALL|STORE|BOUTIQUE|MARKET', case=False, na=False),
        df['STATION_NAME'].str.contains('null', case=False, na=False),
        df['STATION_NAME'].isna()
    ],
    [
        'STREET',
        'AVENUE',
        'ROAD/PKWY/HIGHWAY/BLVD/LANE',
        'SQUARE/PLAZA/CENTER/TERMINAL/CIRCLE/COURT',
        'HALL/BRIDGE/BUILDING/PIER/TUNNEL/HILL',
        'PARK/GARDEN/PARKWAY',
        'STATION/AIRPORT',
        'OFFICE',
        'COLLEGE/UNIVERSITY/SCHOOL',
        'HOSPITAL/CLINIC/MEDICAL/HEALTH',
        'MUSEUM/AQUARIUM/GALLERY/EXHIBITION/CULTURE',
        'SHOPPING/MALL/STORE/BOUTIQUE/MARKET',
        'UNKNOWN',
        'UNKNOWN'
    ],
    default='Other'
)

Cleans and formats station names

In [24]:
date_columns = ["CMPLNT_FR_DT", "CMPLNT_TO_DT"]
for date_col in date_columns:
    if date_col in df.columns:
        df = df[(df[date_col].dt.year >= 2006) & (df[date_col].dt.year <= 2019)]

Filter dates between 2006 and 2019

In [25]:
if 'Latitude' in df.columns and 'Longitude' in df.columns:
    df = df[(df['Latitude'] >= 40.477399) & (df['Latitude'] <= 40.917577)]
    df = df[(df['Longitude'] >= -74.25909) & (df['Longitude'] <= -73.700272)]

Remove rows with invalid latitude and longitude for New York City


In [26]:
invalid_values = ["UNKNOWN", "(null)", "-1", ""]
valid_columns = ['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'OFNS_DESC', 'LAW_CAT_CD', 'BORO_NM', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'Latitude', 'Longitude']

for column in valid_columns:
    if df[column].dtype == object:
        df = df[~df[column].isin(invalid_values)]
    elif df[column].dtype in [np.int64, np.float64]:
        df = df[df[column] != -1]
        
df.dropna(inplace=True, subset=valid_columns)

Remove rows with invalid values for each relevant column

## 4. Analysing Data:

In [27]:
print("First 10 rows of the data:")
print(df.head(10))

First 10 rows of the data:
   CMPLNT_NUM CMPLNT_FR_DT        CMPLNT_FR_TM CMPLNT_TO_DT  \
0    39468181   2008-02-20 2024-06-11 07:00:00   2008-02-23   
1    50539499   2008-08-21 2024-06-11 22:00:00   2008-08-21   
2    45223390   2008-04-03 2024-06-11 03:35:00   2008-04-03   
4    44451016   2008-03-10 2024-06-11 22:00:00   2008-03-10   
5    49918184   2008-08-08 2024-06-11 01:30:00   2008-08-08   
8    47391386   2008-06-19 2024-06-11 15:30:00   2008-06-19   
9    45814265   2008-04-19 2024-06-11 19:45:00   2008-04-19   
10   47131410   2008-06-08 2024-06-11 11:00:00   2008-06-09   
11   45672193   2008-04-13 2024-06-11 17:00:00   2008-04-14   
12   37570315   2008-01-07 2024-06-11 02:00:00   2008-01-07   

          CMPLNT_TO_TM  ADDR_PCT_CD            OFNS_DESC  PD_CD   LAW_CAT_CD  \
0  2024-06-11 08:00:00           88      PROPERTY CRIMES    221       FELONY   
1  2024-06-11 23:00:00           19      PROPERTY CRIMES    438       FELONY   
2  2024-06-11 03:50:00           77    

Prints first 10 rows

In [28]:
print("Shape of the data:", df.shape)

Shape of the data: (4183078, 24)


Prints shape of the data

In [29]:
print("Columns in the data:", df.columns)

Columns in the data: Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'ADDR_PCT_CD', 'OFNS_DESC', 'PD_CD', 'LAW_CAT_CD',
       'BORO_NM', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'HOUSING_PSA',
       'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'Latitude', 'Longitude',
       'Lat_Lon', 'PATROL_BORO', 'STATION_NAME', 'VIC_AGE_GROUP', 'VIC_RACE',
       'VIC_SEX'],
      dtype='object')


Prints columns of the data

In [30]:
print("Descriptive statistics of the data:")
print(df.describe())

Descriptive statistics of the data:
                        CMPLNT_FR_DT                   CMPLNT_FR_TM  \
count                        4183078                        4183078   
mean   2013-04-03 20:13:35.541857280  2024-06-11 13:31:47.065542656   
min              2006-01-01 00:00:00            2024-06-11 00:00:00   
25%              2009-10-10 00:00:00            2024-06-11 09:00:00   
50%              2013-06-05 00:00:00            2024-06-11 14:30:00   
75%              2016-10-07 00:00:00            2024-06-11 19:00:00   
max              2019-12-31 00:00:00            2024-06-11 23:59:00   
std                              NaN                            NaN   

                        CMPLNT_TO_DT                   CMPLNT_TO_TM  \
count                        4183078                        4183078   
mean   2013-04-10 18:25:05.527555584  2024-06-11 13:31:40.666662912   
min              2006-01-01 00:00:00            2024-06-11 00:00:00   
25%              2009-10-17 00:00:00    

Prints descriptive statistics

In [31]:
print("Info about the data:")
df.info()

Info about the data:
<class 'pandas.core.frame.DataFrame'>
Index: 4183078 entries, 0 to 8914837
Data columns (total 24 columns):
 #   Column             Dtype         
---  ------             -----         
 0   CMPLNT_NUM         object        
 1   CMPLNT_FR_DT       datetime64[ns]
 2   CMPLNT_FR_TM       datetime64[ns]
 3   CMPLNT_TO_DT       datetime64[ns]
 4   CMPLNT_TO_TM       datetime64[ns]
 5   ADDR_PCT_CD        int64         
 6   OFNS_DESC          object        
 7   PD_CD              int64         
 8   LAW_CAT_CD         object        
 9   BORO_NM            object        
 10  LOC_OF_OCCUR_DESC  object        
 11  PREM_TYP_DESC      object        
 12  HOUSING_PSA        int64         
 13  SUSP_AGE_GROUP     object        
 14  SUSP_RACE          object        
 15  SUSP_SEX           object        
 16  Latitude           float64       
 17  Longitude          float64       
 18  Lat_Lon            object        
 19  PATROL_BORO        object        
 20  STATION_

Prints information about the data

In [32]:
print("Data types of each column:")
for col in df.columns:
    t = set()
    for item in df[col]:
        t.add(type(item))
    print(f"Column '{col}' has types: {t}")

Data types of each column:
Column 'CMPLNT_NUM' has types: {<class 'str'>}
Column 'CMPLNT_FR_DT' has types: {<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
Column 'CMPLNT_FR_TM' has types: {<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
Column 'CMPLNT_TO_DT' has types: {<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
Column 'CMPLNT_TO_TM' has types: {<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
Column 'ADDR_PCT_CD' has types: {<class 'int'>}
Column 'OFNS_DESC' has types: {<class 'str'>}
Column 'PD_CD' has types: {<class 'int'>}
Column 'LAW_CAT_CD' has types: {<class 'str'>}
Column 'BORO_NM' has types: {<class 'str'>}
Column 'LOC_OF_OCCUR_DESC' has types: {<class 'str'>}
Column 'PREM_TYP_DESC' has types: {<class 'str'>}
Column 'HOUSING_PSA' has types: {<class 'int'>}
Column 'SUSP_AGE_GROUP' has types: {<class 'float'>, <class 'str'>}
Column 'SUSP_RACE' has types: {<class 'float'>, <class 'str'>}
Column 'SUSP_SEX' has types: {<class 'float'>, <class 'str'>}
Column 'La

Prints data types of each column

In [33]:
print("Unique values in each column:")
for col in df.columns:
    print(f"Column '{col}' unique values:", df[col].unique())

Unique values in each column:
Column 'CMPLNT_NUM' unique values: ['39468181' '50539499' '45223390' ... '52614163' '47521554' '51084712']
Column 'CMPLNT_FR_DT' unique values: <DatetimeArray>
['2008-02-20 00:00:00', '2008-08-21 00:00:00', '2008-04-03 00:00:00',
 '2008-03-10 00:00:00', '2008-08-08 00:00:00', '2008-06-19 00:00:00',
 '2008-04-19 00:00:00', '2008-06-08 00:00:00', '2008-04-13 00:00:00',
 '2008-01-07 00:00:00',
 ...
 '2006-08-13 00:00:00', '2006-04-17 00:00:00', '2006-10-11 00:00:00',
 '2006-08-29 00:00:00', '2006-04-21 00:00:00', '2007-03-22 00:00:00',
 '2007-02-18 00:00:00', '2007-04-12 00:00:00', '2007-08-23 00:00:00',
 '2007-03-09 00:00:00']
Length: 5113, dtype: datetime64[ns]
Column 'CMPLNT_FR_TM' unique values: <DatetimeArray>
['2024-06-11 07:00:00', '2024-06-11 22:00:00', '2024-06-11 03:35:00',
 '2024-06-11 01:30:00', '2024-06-11 15:30:00', '2024-06-11 19:45:00',
 '2024-06-11 11:00:00', '2024-06-11 17:00:00', '2024-06-11 02:00:00',
 '2024-06-11 19:00:00',
 ...
 '2024-06

Prints unique values for each column

In [34]:
print("Percentage of missing values for each column:")
missing_percentage = (df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False)
print(missing_percentage, "\n")

Percentage of missing values for each column:
SUSP_AGE_GROUP       58.400991
SUSP_SEX             57.386977
SUSP_RACE            46.213817
VIC_SEX              24.851270
VIC_AGE_GROUP        19.634561
VIC_RACE              0.003897
CMPLNT_FR_DT          0.000000
STATION_NAME          0.000000
PATROL_BORO           0.000000
Lat_Lon               0.000000
Longitude             0.000000
Latitude              0.000000
CMPLNT_NUM            0.000000
PREM_TYP_DESC         0.000000
LOC_OF_OCCUR_DESC     0.000000
BORO_NM               0.000000
LAW_CAT_CD            0.000000
PD_CD                 0.000000
OFNS_DESC             0.000000
ADDR_PCT_CD           0.000000
CMPLNT_TO_TM          0.000000
CMPLNT_TO_DT          0.000000
CMPLNT_FR_TM          0.000000
HOUSING_PSA           0.000000
dtype: float64 



Prints the percentage of missing values for each column

## 5. Defining Weights for Crime Categories:

In [35]:
weights = {'FELONY': 2.5, 'MISDEMEANOR': 1.7, 'VIOLATION': 1}

'weights': A dictionary mapping crime categories to their respective weights.

## 6. Mapping Weights to the DataFrame:

In [36]:
df['weighted_crime_count'] = df['LAW_CAT_CD'].map(weights)

Adds a new column weighted_crime_count to the DataFrame df by mapping the LAW_CAT_CD values to the predefined weights.

## 7. Aggregating Data by Time and Precinct:

In [37]:
df['CMPLNT_FR_DT'] = pd.to_datetime(df['CMPLNT_FR_DT'])
df['CMPLNT_FR_TM'] = df['CMPLNT_FR_TM'].apply(lambda x: x.hour)

In [38]:
agg_df = df.groupby(['CMPLNT_FR_TM'] + [col for col in df.columns if 'ADDR_PCT_CD' in col])['weighted_crime_count'].sum().reset_index()

Prepares CMPLNT_FR_DT and CMPLNT_FR_TM columns for further usage.

Groups the data by CMPLNT_FR_TM and precinct columns, summing the weighted_crime_count for each group.

## 8. Defining Danger Levels:

In [39]:
threshold_low = agg_df['weighted_crime_count'].quantile(0.30)
threshold_high = agg_df['weighted_crime_count'].quantile(0.70)

agg_df['danger_level'] = pd.cut(agg_df['weighted_crime_count'], bins=[0, threshold_low, threshold_high, float('inf')], labels=['low', 'mid', 'high'])

Calculates the 30th and 70th percentiles of weighted_crime_count.

Assigns danger levels (low, mid, high) based on these thresholds.

## 9. Preparing Features and Labels:

In [40]:
X = agg_df.drop(columns=['danger_level', 'weighted_crime_count'])
y = agg_df['danger_level']

X: Features for the model (all columns except danger_level and weighted_crime_count).

y: Labels for the model (the danger_level column).

## 10. Splitting the Data:

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Splits the data into training and testing sets (80% train, 20% test).

## 11. Defining Hyperparameter Grid:

In [42]:
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

Defines a grid of hyperparameters for the Gradient Boosting Classifier.

## 12. Initializing and Fitting GridSearchCV:

In [43]:
grid_search = GridSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [100, 200]},
             verbose=2)

Initializes GridSearchCV with the Gradient Boosting Classifier and the hyperparameter grid.

Fits GridSearchCV to the training data to find the best hyperparameters.

## 13. Outputting Best Parameters and Score:

In [44]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Best score: 0.8089430894308943


Prints the best hyperparameters and the corresponding score.

## 14. Predicting and Evaluating the Model:

In [45]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        high       0.84      0.86      0.85       100
         low       0.83      0.84      0.83       119
         mid       0.78      0.76      0.77       151

    accuracy                           0.81       370
   macro avg       0.82      0.82      0.82       370
weighted avg       0.81      0.81      0.81       370



Uses the best model from GridSearchCV to make predictions on the test set.
Prints the classification report to evaluate model performance.